# AIS Management GmbH – Technische Anlagenanalyse

**Ziele:**
- Vollständigkeitsprüfung der Kundendatei anhand von Beispielobjekten
- Zuordnung von Artikelnummern aus dem EP-Katalog


In [1]:
# Import libraries
import pandas as pd
import difflib
from collections import defaultdict

In [2]:
# Import excel data
kundendatei = pd.read_excel("Kundendatei.xlsx", sheet_name="Kundenobjekte")
ep_katalog = pd.read_excel("EP_Katalog.xlsx", sheet_name="EP Liste")
beispielobjekte = pd.read_excel("Beispielobjekte.xlsx", sheet_name="Anlagen")

In [3]:
# Clean up data
kundendatei = kundendatei[["WirtEinh", "EQ-Bezeichnung", "EQ-Klasse-Bezeichnung", "EQ-Menge"]].dropna(subset=["EQ-Bezeichnung"])
kundendatei.columns = ["Gebaeude", "Anlagenname", "Klasse", "Menge"]
kundendatei["Anlagenname_clean"] = kundendatei["Anlagenname"].str.lower().str.strip()

beispielobjekte = beispielobjekte[["Gebäude-ID", "Anlagenname", "Kostengruppenbezeichnung"]].dropna(subset=["Anlagenname"])
beispielobjekte.columns = ["Gebaeude", "Anlagenname", "Kategorie"]
beispielobjekte["Anlagenname_clean"] = beispielobjekte["Anlagenname"].str.lower().str.strip()

ep_katalog = ep_katalog[["Kurztext / Bezeichnung", "Artikelnummer"]].dropna()
ep_katalog.columns = ["Bezeichnung", "Artikelnummer"]
ep_katalog["Bezeichnung_clean"] = ep_katalog["Bezeichnung"].str.lower().str.strip()

In [4]:
# completeness check
reference_systems = set(beispielobjekte["Anlagenname_clean"].unique())
missing_by_building = defaultdict(list)

for geb in kundendatei["Gebaeude"].unique():
    actual = set(kundendatei[kundendatei["Gebaeude"] == geb]["Anlagenname_clean"])
    missing = reference_systems - actual
    missing_by_building[geb] = list(missing)

# Example output
for geb, fehlend in list(missing_by_building.items())[:3]:
    print(f"Gebäude {geb} fehlt {len(fehlend)} Anlagen, z.B.:", fehlend[:5])

Gebäude 21083 fehlt 3269 Anlagen, z.B.: ['ns-unterverteiler aufzug 1', 'zirkulationspumpe', 'brandschutztüre stg. 1 4og', 'handfeuerlöscher 81', 'bst 3og stiegenhaus']
Gebäude 28580 fehlt 3272 Anlagen, z.B.: ['ns-unterverteiler aufzug 1', 'zirkulationspumpe', 'brandschutztüre stg. 1 4og', 'handfeuerlöscher 81', 'bst 3og stiegenhaus']
Gebäude 22225 fehlt 3272 Anlagen, z.B.: ['ns-unterverteiler aufzug 1', 'zirkulationspumpe', 'brandschutztüre stg. 1 4og', 'handfeuerlöscher 81', 'bst 3og stiegenhaus']


In [5]:
# EP assignment (best match)
catalog_names = ep_katalog["Bezeichnung_clean"].tolist()
matches = []

for _, row in kundendatei.iterrows():
    name = row["Anlagenname_clean"]
    best_match = difflib.get_close_matches(name, catalog_names, n=1, cutoff=0.6)
    if best_match:
        match_row = ep_katalog[ep_katalog["Bezeichnung_clean"] == best_match[0]].iloc[0]
        matches.append({
            "Gebaeude": row["Gebaeude"],
            "Anlage": row["Anlagenname"],
            "EP-Bezeichnung": match_row["Bezeichnung"],
            "Artikelnummer": match_row["Artikelnummer"]
        })
    else:
        matches.append({
            "Gebaeude": row["Gebaeude"],
            "Anlage": row["Anlagenname"],
            "EP-Bezeichnung": "Keine Übereinstimmung",
            "Artikelnummer": "-"
        })

ep_zuordnung_df = pd.DataFrame(matches)
ep_zuordnung_df.to_excel("EP_Zuordnung_Ergebnis.xlsx", index=False)
print("Results saved as : EP_Zuordnung_Ergebnis.xlsx")

✅ Ergebnis gespeichert unter: EP_Zuordnung_Ergebnis.xlsx
